In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import os
import random
from scipy.misc import imsave, imresize
import warnings

df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Skill Moves,SlidingTackle,SprintSpeed,Stamina,StandingTackle,Strength,Vision,Volleys,Weak Foot,Work Rate
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,4,26,86,72,28,59,94,86,4,Medium/ Medium
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,5,23,91,88,31,79,82,87,4,High/ Low
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,5,33,90,81,24,49,87,84,5,High/ Medium
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,1,13,58,43,21,64,68,13,3,Medium/ Medium
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,4,51,76,90,58,75,94,82,5,High/ High


In [6]:
#Read from csv file, get aggression rating

df_copy = df.copy()
useful_data = {}
for player in df_copy.itertuples(index = True, name='Pandas'):
    #print()
    details = []
    p_id = getattr(player,"ID")
    name = getattr(player,"Name")
    url = list(getattr(player,"Photo"))
    url[31] = '10'
    photo_url = "".join(url)
    aggression = getattr(player,"Aggression")
    rating = getattr(player,"Overall")
    details.append(name)
    details.append(photo_url)
    details.append(aggression)
    details.append(rating)
    useful_data[p_id] = details
    
print(len(useful_data))



17790


In [15]:
def get_images(url,name):
    res = requests.get(url)
    if res.ok:
        img_data = res.content
        save_name = name + '.jpg'
        #remove comments to download the pictures
        with open(os.path.join('faces',save_name),'wb') as handler:
            handler.write(img_data)
        #print("success")
        return save_name 
    else:
        #print("invalid?")
        return "invalid"
     



In [8]:
def preprocessing(photo_dict):
    training_split = {}
    test_split = {}
    test_size = 0.2 * len(photo_dict)
    for key in photo_dict:
        if len(test_split) < test_size:
            k = random.randint(0,1)
            if k == 1:
                test_split[key] = photo_dict[key]
            else: training_split[key] = photo_dict[key]
        else: training_split[key] = photo_dict[key]
    print(len(training_split))
    print(len(test_split))
    return training_split,test_split

[0 0 0 1]


In [10]:


#training_data = get_training_data(training_s)
#print(shape(training_data))

In [16]:
counter = 0
for key in useful_data:
    content = useful_data[key]
    
    name = str(key)
    #print("downloading",name,"'s picture")
    url = str(content[1])
    image_path = get_images(url,name)
    content.append(image_path)
    counter += 1
    if counter % 1000 == 0:
        print(counter)
print("done")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
done


In [17]:
print(len(useful_data))
#delete entries without photos
def del_invalid(photo_dict):
    delete_list = [key for key in photo_dict if photo_dict[key][-1] is "invalid"]
    for key in delete_list:
        del photo_dict[key]
    return photo_dict
useful_data = del_invalid(useful_data)
print(len(useful_data))

15096
15096


In [18]:
training_s,test_s = preprocessing(useful_data)

12076
3020


In [26]:
#create csv files for both training and test sets
test_copy = df.copy()
training_csv = [p_id for p_id in training_s]
test_csv = [p_id for p_id in test_s]

for index,row in df_copy.iterrows():
    if row['ID'] not in training_csv:
        df_copy.drop(index,inplace=True)
print(len(df_copy))
print(len(test_copy))
for index2,row2 in test_copy.iterrows():
    if row2['ID'] not in test_csv:
        test_copy.drop(index2,inplace=True)
df_copy.to_csv('training_set.csv', encoding='utf-8',index= False)
test_copy.to_csv('test_set.csv', encoding='utf-8',index= False)
print(len(test_copy))
print("done")

12076
17790
3020
done


12076
